In [ ]:
import base64

def encode(s):
    return ('data:text/javascript;base64,'
            + base64.standard_b64encode(s.encode('utf-8')).decode('utf-8'))

## Opening a Comm from `context.comms` returns a non-null object

In [ ]:
display({
    'application/vnd.jupyter.es6-rich-output': encode('''
    export function render(output, element, context) {
        (async () => {
            var comm = await context.comms.open('test-id', {});
            var resultElement = document.createElement("p")
            if (comm) {
                resultElement.textContent = "Successfully opened comm"
                resultElement.style.color = 'green'
            } else {
                resultElement.textContent = "Failed to open comm"
                resultElement.style.color = 'red'
            }   
            element.appendChild(resultElement)
        })()
        
    }
    '''),
    'application/mytable+json': [[1,2,3],[4,5,6]]
}, raw=True)


## Registering Comm in Python, open in rich output

In [ ]:
import IPython

ipython = IPython.get_ipython()
messages = []

def on_comm_creation(comm, open_msg):
    @comm.on_msg
    def _recv(msg):
        messages.append(msg)
        
    comm.send({'echo': open_msg['content']['data']})

# Register the comm target - so the callback gets called
ipython.kernel.comm_manager.register_target('comm-test-target-id', on_comm_creation)

In [ ]:
display({
    'application/vnd.jupyter.es6-rich-output': encode('''
    export function render(output, element, context) {
        (async () => {
            var comm = await context.comms.open('comm-test-target-id', {});
            await comm.send("Test Message 1")
            await comm.send("Test Message 2")
            await comm.send("Test Message 3")
            
            var resultElement = document.createElement("p")
            resultElement.textContent = "Sent Comm Message"
            resultElement.style.color = 'green'
            element.appendChild(resultElement);
            
            
            
            
            console.log(comm.messages)
        })()
    
    }
    '''),
}, raw=True)


In [ ]:
assert len(messages) == 3 # run from start of section to reset messages if this fails